In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
#from sklearn import metrics
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split 
# from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

In [5]:
# site = data.loc[data['Site Number'] == 2001]

# y_train = site['pctChgExclShopSalesFromMedian']
# x_train = site['AdoApprPriceChgPct']
# #x_train = sm.add_constant(x_train)
# model = sm.OLS(y_train, x_train).fit()

# model.params[0] # gives the coefficients

# model.pvalues[0] #model.tvalues[0] # model.rsquared_adj[0]    #model.rsquared

# #X_train = sm.add_constant(X_train)
# model = sm.OLS(y_train, X_train).fit()
# params.append(dict(model.params))

In [41]:
model.params

AdoApprPriceChgPct   -3.02
dtype: float64

In [49]:
prefix = 'ADO_'

In [50]:
cols = ['A','B','C']

In [54]:
list(map(lambda x: prefix+x,cols))

['ADO_A', 'ADO_B', 'ADO_C']

In [57]:
#elasticity calculation :: forcing the intercept to zero 
def regRes_gen_no_const(site,x,y,prefix):
    # filter all data of the site between 5th & 95th precentile 
    q_low,q_up = site[y].quantile([0.02,0.98]).values.round().tolist()
    xf = site.loc[(site[y] >= q_low)& (site[y] <= q_up)]
    
    q_low,q_up = site[x].quantile([0.02,0.98]).values.round().tolist()
    xf = site.loc[(site[x] >= q_low)& (site[x] <= q_up)]
    
    model = sm.OLS(xf[y],xf[x]).fit()
    slope,rvalu,pvalue = model.params[0],round(model.rsquared,2),model.pvalues[0]
    #slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf[x],xf[y])
    cols = ['slope','rvalu','pvalue','count']
    cols_ref = list(map(lambda x: x+prefix,cols))
    return(pd.Series([slope,rvalu,pvalue,len(xf)],index=cols_ref))

In [64]:
# calculation with respect to ADO price position 
# exclusive shop sales 
x_var = 'AdoApprPriceChgPct'
ref = '_ADO'
regExclShopSalesVsAppPriceADO_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgExclShopSalesFromMedian',ref)).reset_index()
# shop sales from fuel sales
regShopSalesFromFuelVsAppPriceADO_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgShopSalesFromFuelFromMedian',ref)).reset_index()
# fuel sales
regFuelSalesVsAppPriceADO_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgFuelSalesFromMedian',ref)).reset_index()

# calculation with respect to UL 95 price position 
# exclusive shop sales 
x_var = 'UL95AppPriceChgPct'
ref = '_UL95'
regExclShopSalesVsAppPriceUL95_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgExclShopSalesFromMedian',ref)).reset_index()
# shop sales from fuel sales
regShopSalesFromFuelVsAppPriceUL95_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgShopSalesFromFuelFromMedian',ref)).reset_index()
# fuel sales
regFuelSalesVsAppPriceUL95_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgFuelSalesFromMedian',ref)).reset_index()

In [84]:
# reqColumns = ['Site Number', 'Site Name', 'Site ID', 'PBL','Transaction Date', 'Total Sales Value',\
#                'pctChgExclShopSalesFromMedian','pctChgShopSalesFromFuelFromMedian',\
#                 'pctChgFuelSalesFromMedian','AdoApprPriceChgPct', 'UL95AppPriceChgPct',\
#                'UL 95_ Approved Price Vs Marker Price','UL 95_ Approved Price Vs Marker Price',\
#                'Total Sales Value','exclShopSales','shopSalesFromFuel','totalSalesEsso']
# data.loc[:,reqColumns].to_excel('data_tableau_elasticity.xlsx',index=False)

In [65]:
regFuelSalesVsAppPriceADO_pct.head()

Site Name  slope_ADO  rvalu_ADO  pvalue_ADO  count_ADO
0  ABBEY ST ACCRNGTN EXP      -1.95       0.03        0.00     302.00
1       ABINGTON EXPRESS      -1.75       0.01        0.20     309.00
2  ACKLAM RD MIDSBRO EXP      -2.51       0.05        0.00     282.00
3          ACTON EXPRESS       0.31       0.00        0.71     248.00
4     ADDISCOMBE EXPRESS      -1.66       0.02        0.02     301.00

In [66]:
regFuelSalesVsAppPriceADO_pct['el_q'] = pd.qcut(regFuelSalesVsAppPriceADO_pct.slope_ADO,5)

regFuelSalesVsAppPriceADO_pct['el_q'].value_counts()

[-10.817, -2.283]    40
(0.139, 12.613]      39
(-0.747, 0.139]      39
(-1.509, -0.747]     39
(-2.283, -1.509]     39
Name: el_q, dtype: int64

In [67]:
bins_val = [-11,-5,-2.3,-1.5,-0.75,0.15,5,20] 
bin_labels = ['Very High','High','Medium','Low','Very Low','positive','High positive']
regFuelSalesVsAppPriceADO_pct['el_levels_ADO'] = pd.cut(regFuelSalesVsAppPriceADO_pct['slope_ADO'], bins = bins_val,
                                   labels=bin_labels,include_lowest=True)

regFuelSalesVsAppPriceADO_pct.groupby('el_levels_ADO')['slope_ADO'].describe().unstack().reset_index()

el_levels_ADO  count  mean  std    min   25%   50%   75%   max
0      Very High   6.00 -6.92 2.14 -10.82 -7.49 -6.19 -5.53 -5.07
1           High  32.00 -3.32 0.59  -4.54 -3.65 -3.18 -2.82 -2.40
2         Medium  42.00 -1.84 0.25  -2.30 -2.07 -1.77 -1.62 -1.51
3            Low  36.00 -1.12 0.22  -1.49 -1.31 -1.11 -0.91 -0.79
4       Very Low  41.00 -0.40 0.22  -0.75 -0.56 -0.39 -0.27  0.14
5       positive  37.00  1.29 1.05   0.21  0.46  0.99  1.63  3.95
6  High positive   2.00 10.45 3.06   8.28  9.36 10.45 11.53 12.61

In [58]:
PDI_elasticity_results = pd.read_excel(r'PDI_elasticity_results.xlsx')

In [63]:
PDI_elasticity_results.head()

Site Id                                 Site Elasticity UL  EL UL  \
0   200005     TESCO BOLTON EXPRESS (RT) BOLTON       4 - Low     -6   
1   200012        TESCO OVENDEN HALIFAX EXPRESS  5 - Very Low     -5   
2   200028      TESCO YATELEY EXPRESS (RT) FUEL      2 - High     -8   
3   200040  TESCO HESKETH SOUTHPORT EXPRESS(RT)       4 - Low     -6   
4   200043   TESCO CHURCHTOWN EXPRESS (RT) FUEL  5 - Very Low     -5   

  Elasticity ADO  EL ADO  EL TOTAL       From                   To  
0     3 - Medium      -7     -6.50 2019-08-05  2999-12-31 23:59:59  
1   5 - Very Low      -5     -5.00 2019-08-05  2999-12-31 23:59:59  
2       2 - High      -8     -8.00 2019-08-05  2999-12-31 23:59:59  
3     3 - Medium      -7     -6.50 2019-08-05  2999-12-31 23:59:59  
4     3 - Medium      -7     -6.00 2019-08-05  2999-12-31 23:59:59

In [62]:
PDI_elasticity_results['EL ADO'].describe()

count   196.00
mean     -5.80
std       0.83
min      -9.00
25%      -6.00
50%      -6.00
75%      -5.00
max      -5.00
Name: EL ADO, dtype: float64

In [68]:
PDI_elasticity_results['Elasticity ADO'].value_counts()

5 - Very Low     82
4 - Low          80
3 - Medium       26
2 - High          6
1 - Very High     1
Name: Elasticity ADO, dtype: int64

In [69]:
site_id_name = data.loc[:,['PBL', 'Site Name']].drop_duplicates()
regFuelSalesVsAppPriceADO_pct = regFuelSalesVsAppPriceADO_pct.merge(site_id_name,how = 'left',on='Site Name')
#regFuelSalesVsAppPriceADO_pct.to_excel(r'fuelElasticity_regression.xlsx')
regFuelSalesVsAppPrice = regFuelSalesVsAppPriceADO_pct.merge(regFuelSalesVsAppPriceUL95_pct,how = 'left',on='Site Name')
#PDI_elasticity_results = pd.read_excel(r'PDI_elasticity_results.xlsx')
regFuelSalesVsAppPrice_pdi = regFuelSalesVsAppPrice.merge(PDI_elasticity_results,\
                        how = 'left',left_on='PBL',right_on='Site Id')

In [105]:
regFuelSalesVsAppPrice_pdi

Site Name  slope_ADO  rvalu_ADO  pvalue_ADO  count_ADO  \
0    ABBEY ST ACCRNGTN EXP      -1.95       0.03        0.00     302.00   
1         ABINGTON EXPRESS      -1.75       0.01        0.20     309.00   
2    ACKLAM RD MIDSBRO EXP      -2.51       0.05        0.00     282.00   
3            ACTON EXPRESS       0.31       0.00        0.71     248.00   
4       ADDISCOMBE EXPRESS      -1.66       0.02        0.02     301.00   
5          ARDSLEY EXPRESS      -3.58       0.11        0.00     290.00   
6        ARDWICK GREEN EXP      -3.87       0.07        0.00     342.00   
7    ASHTON IN MAKRFLD EXP       0.30       0.00        0.62     260.00   
8           AULD CROSS EXP      -0.89       0.01        0.18     299.00   
9           BALHAM EXPRESS       1.54       0.01        0.17     303.00   
10         BARKING EXPRESS      -0.26       0.00        0.83     275.00   
11          BARNES EXPRESS      12.61       0.08        0.00     311.00   
12         BASFORD EXPRESS      -1.95       0.03        0.00     282.00   
13          BATTLE EXPRESS       1.90       0.01        0.25     133.00   
14        BEARSTED EXPRESS      -5.07       0.15        0.00     322.00   
15   BESSACARR DONCAST EXP       1.68       0.08        0.00     300.00   
16   BEWDLEY HILL KIDD EXP      -1.01       0.02        0.01     322.00   
17      BILLY MILL EXPRESS      -3.59       0.05        0.00     315.00   
18        BITTERNE EXPRESS      -1.21       0.01        0.11     283.00   
19   BOLTON BRADFRD ST EXP      -0.56       0.00        0.37     290.00   
20   BOOTHFERRY HESSLE EXP      -3.08       0.04        0.00     350.00   
21          BOURNE EXPRESS      -1.52       0.01        0.05     289.00   
22     BOURNVILLE BHAM EXP      -2.28       0.02        0.01     297.00   
23   BRAMHALL CHESHIRE EXP      -0.24       0.00        0.79     275.00   
24         BRECHIN EXPRESS      -1.37       0.03        0.00     249.00   
25       BRENTWOOD EXPRESS       0.52       0.00        0.58     307.00   
26     BRISTL SOUNDWEL EXP      -0.88       0.01        0.14     304.00   
27      BRISTOL FILTON EXP       0.14       0.00        0.80     320.00   
28         BROMLEY EXPRESS      -2.13       0.01        0.06     301.00   
29   BULLSMRE LN WAL X EXP      -0.21       0.00        0.81     309.00   
..                     ...        ...        ...         ...        ...   
166    STORRINGTON EXPRESS      -3.07       0.05        0.00     325.00   
167  TARBOCK WOODLANDS EXP       3.69       0.11        0.00     240.00   
168       TOLWORTH EXPRESS      -1.02       0.01        0.12     292.00   
169       TONBRIDGE RD EXP      -1.87       0.01        0.03     318.00   
170        TORQUAY EXPRESS      -5.93       0.06        0.00     304.00   
171      TULSEHILL EXPRESS      -0.35       0.00        0.51     303.00   
172        TUPSLEY EXPRESS      -2.19       0.02        0.02     321.00   
173  TWYFORD RD EASTLG EXP      -2.07       0.08        0.00     305.00   
174    WALTHAMSTOW EXPRESS      -2.08       0.01        0.09     297.00   
175            WARMLEY EXP      -0.70       0.01        0.09     287.00   
176    WEDNESFIELD EXPRESS      -0.39       0.00        0.57     286.00   
177   WEST BYFLEET EXPRESS      -0.80       0.01        0.31     203.00   
178   WEST WICKHAM EXPRESS      -2.77       0.05        0.00     294.00   
179           WESTBURN EXP       0.28       0.00        0.60     283.00   
180  WEYMOUTH LANEH RD EXP      -2.60       0.01        0.05     318.00   
181     WHITCHURCH EXPRESS       0.55       0.00        0.31     278.00   
182     WHITEGATES EXPRESS      -0.47       0.00        0.46     309.00   
183  WHITEHOUS SCOLDFD EXP       0.82       0.00        0.21     330.00   
184    WIGAN RD BOLTON EXP       1.90       0.02        0.03     331.00   
185    WILLENHALL BHAM EXP       0.65       0.00        0.23     292.00   
186      WIMBLEDON EXPRESS     -10.82       0.17        0.00     297.00   
187  WINCHMORE HILL EXPRES      -2.30     

In [110]:
selCols = ['Site Name', 'slope_ADO','el_levels_ADO','EL ADO','Elasticity ADO','rvalu_ADO', 'pvalue_ADO',\
           'slope_UL95', 'EL UL' ,'Elasticity UL','EL TOTAL',\
          'rvalu_UL95','pvalue_UL95','Site Id', 'Site','Site Number', 'PBL']
df_el = regFuelSalesVsAppPrice_pdi.loc[:,selCols]
df_el.to_excel('elasticity_calcuation_PDI_vs_regression_2.xlsx',index=False)

In [109]:
df_el

Site Name  slope_ADO  el_levels_ADO  EL ADO Elasticity ADO  \
0    ABBEY ST ACCRNGTN EXP      -1.95         Medium   -7.00     3 - Medium   
1         ABINGTON EXPRESS      -1.75         Medium   -5.00   5 - Very Low   
2    ACKLAM RD MIDSBRO EXP      -2.51           High   -5.00   5 - Very Low   
3            ACTON EXPRESS       0.31       positive   -7.00     3 - Medium   
4       ADDISCOMBE EXPRESS      -1.66         Medium   -7.00     3 - Medium   
5          ARDSLEY EXPRESS      -3.58           High   -5.00   5 - Very Low   
6        ARDWICK GREEN EXP      -3.87           High   -6.00        4 - Low   
7    ASHTON IN MAKRFLD EXP       0.30       positive   -5.00   5 - Very Low   
8           AULD CROSS EXP      -0.89            Low   -6.00        4 - Low   
9           BALHAM EXPRESS       1.54       positive   -5.00   5 - Very Low   
10         BARKING EXPRESS      -0.26       Very Low   -6.00        4 - Low   
11          BARNES EXPRESS      12.61  High positive   -6.00        4 - Low   
12         BASFORD EXPRESS      -1.95         Medium   -6.00        4 - Low   
13          BATTLE EXPRESS       1.90       positive   -6.00        4 - Low   
14        BEARSTED EXPRESS      -5.07      Very High   -5.00   5 - Very Low   
15   BESSACARR DONCAST EXP       1.68       positive   -5.00   5 - Very Low   
16   BEWDLEY HILL KIDD EXP      -1.01            Low   -6.00        4 - Low   
17      BILLY MILL EXPRESS      -3.59           High   -6.00        4 - Low   
18        BITTERNE EXPRESS      -1.21            Low   -7.00     3 - Medium   
19   BOLTON BRADFRD ST EXP      -0.56       Very Low   -7.00     3 - Medium   
20   BOOTHFERRY HESSLE EXP      -3.08           High   -6.00        4 - Low   
21          BOURNE EXPRESS      -1.52         Medium   -6.00        4 - Low   
22     BOURNVILLE BHAM EXP      -2.28         Medium   -7.00     3 - Medium   
23   BRAMHALL CHESHIRE EXP      -0.24       Very Low   -5.00   5 - Very Low   
24         BRECHIN EXPRESS      -1.37            Low   -6.00        4 - Low   
25       BRENTWOOD EXPRESS       0.52       positive   -6.00        4 - Low   
26     BRISTL SOUNDWEL EXP      -0.88            Low   -5.00   5 - Very Low   
27      BRISTOL FILTON EXP       0.14       Very Low   -5.00   5 - Very Low   
28         BROMLEY EXPRESS      -2.13         Medium   -5.00   5 - Very Low   
29   BULLSMRE LN WAL X EXP      -0.21       Very Low   -6.00        4 - Low   
..                     ...        ...            ...     ...            ...   
166    STORRINGTON EXPRESS      -3.07           High     nan            NaN   
167  TARBOCK WOODLANDS EXP       3.69       positive   -5.00   5 - Very Low   
168       TOLWORTH EXPRESS      -1.02            Low   -5.00   5 - Very Low   
169       TONBRIDGE RD EXP      -1.87         Medium   -8.00       2 - High   
170        TORQUAY EXPRESS      -5.93      Very High   -6.00        4 - Low   
171      TULSEHILL EXPRESS      -0.35       Very Low   -6.00        4 - Low   
172        TUPSLEY EXPRESS      -2.19         Medium   -6.00        4 - Low   
173  TWYFORD RD EASTLG EXP      -2.07         Medium   -6.00        4 - Low   
174    WALTHAMSTOW EXPRESS      -2.08         Medium   -6.00        4 - Low   
175            WARMLEY EXP      -0.70       Very Low   -5.00   5 - Very Low   
176    WEDNESFIELD EXPRESS      -0.39       Very Low   -5.00   5 - Very Low   
177   WEST BYFLEET EXPRESS      -0.80            Low   -5.00   5 - Very Low   
178   WEST WICKHAM EXPRESS      -2.77           High   -7.00     3 - Medium   
179           WESTBURN EXP       0.28       positive   -6.00        4 - Low   
180  WEYMOUTH LANEH RD EXP      -2.60           High   -5.00   5 - Very Low   
181     WHITCHURCH EXPRESS       0.55       positive   -5.00   5 - Very Low   
182     WHITEGATES EXPRESS      -0.47       Very Low   -5.00   5 - Very Low   
183  WHITEHOUS SCOLDFD EXP       0.82       positive   -6.00        4 - Low   
184    WIGAN RD BOLTON EXP       1.90       positive   -6.00       

In [107]:
#data.to_pickle('df_tesco_marker_V3.pkl')
df_el.to_pickle('df_el_2.pkl')

In [117]:
regShopSalesFromFuelVsAppPriceADO_pct

Site Name  slope_ADO  rvalu_ADO  pvalue_ADO  count_ADO  \
0    ABBEY ST ACCRNGTN EXP      -0.68       0.00        0.45     302.00   
1         ABINGTON EXPRESS      -4.83       0.01        0.08     309.00   
2    ACKLAM RD MIDSBRO EXP      -0.25       0.00        0.83     282.00   
3            ACTON EXPRESS       2.08       0.02        0.05     248.00   
4       ADDISCOMBE EXPRESS       0.11       0.00        0.91     301.00   
5          ARDSLEY EXPRESS       1.31       0.01        0.15     290.00   
6        ARDWICK GREEN EXP       0.68       0.00        0.57     342.00   
7    ASHTON IN MAKRFLD EXP       0.50       0.00        0.67     260.00   
8           AULD CROSS EXP      -0.73       0.00        0.45     299.00   
9           BALHAM EXPRESS       0.24       0.00        0.90     303.00   
10         BARKING EXPRESS      -3.04       0.01        0.16     275.00   
11          BARNES EXPRESS      10.42       0.07        0.00     311.00   
12         BASFORD EXPRESS      -1.48       0.01        0.16     282.00   
13          BATTLE EXPRESS       5.02       0.06        0.00     133.00   
14        BEARSTED EXPRESS      -2.55       0.02        0.01     322.00   
15   BESSACARR DONCAST EXP       1.45       0.02        0.02     300.00   
16   BEWDLEY HILL KIDD EXP      -0.13       0.00        0.81     322.00   
17      BILLY MILL EXPRESS      -2.35       0.01        0.17     315.00   
18        BITTERNE EXPRESS      -0.08       0.00        0.94     283.00   
19   BOLTON BRADFRD ST EXP       0.53       0.00        0.68     290.00   
20   BOOTHFERRY HESSLE EXP      -1.40       0.00        0.24     350.00   
21          BOURNE EXPRESS      -0.96       0.00        0.43     289.00   
22     BOURNVILLE BHAM EXP       0.54       0.00        0.75     297.00   
23   BRAMHALL CHESHIRE EXP      -1.27       0.00        0.38     275.00   
24         BRECHIN EXPRESS       1.70       0.02        0.04     249.00   
25       BRENTWOOD EXPRESS       1.77       0.00        0.26     307.00   
26     BRISTL SOUNDWEL EXP       0.88       0.00        0.29     304.00   
27      BRISTOL FILTON EXP       0.66       0.00        0.33     320.00   
28         BROMLEY EXPRESS       1.41       0.00        0.49     301.00   
29   BULLSMRE LN WAL X EXP      -0.54       0.00        0.71     309.00   
..                     ...        ...        ...         ...        ...   
166    STORRINGTON EXPRESS       0.10       0.00        0.93     325.00   
167  TARBOCK WOODLANDS EXP       4.93       0.13        0.00     240.00   
168       TOLWORTH EXPRESS      -0.71       0.00        0.68     292.00   
169       TONBRIDGE RD EXP       1.56       0.01        0.16     318.00   
170        TORQUAY EXPRESS      -2.20       0.00        0.33     304.00   
171      TULSEHILL EXPRESS       1.31       0.01        0.11     303.00   
172        TUPSLEY EXPRESS      -3.45       0.01        0.03     321.00   
173  TWYFORD RD EASTLG EXP      -0.93       0.01        0.05     305.00   
174    WALTHAMSTOW EXPRESS       3.23       0.01        0.17     297.00   
175            WARMLEY EXP       1.55       0.03        0.01     287.00   
176    WEDNESFIELD EXPRESS       0.59       0.00        0.61     286.00   
177   WEST BYFLEET EXPRESS       2.84       0.01        0.30     203.00   
178   WEST WICKHAM EXPRESS       0.65       0.00        0.50     294.00   
179           WESTBURN EXP      -0.78       0.00        0.29     283.00   
180  WEYMOUTH LANEH RD EXP       2.06       0.00        0.41     318.00   
181     WHITCHURCH EXPRESS      -0.94       0.00        0.30     278.00   
182     WHITEGATES EXPRESS       1.13       0.00        0.22     309.00   
183  WHITEHOUS SCOLDFD EXP       2.56       0.03        0.00     330.00   
184    WIGAN RD BOLTON EXP       4.50       0.02        0.02     331.00   
185    WILLENHALL BHAM EXP       2.17       0.01        0.05     292.00   
186      WIMBLEDON EXPRESS      -5.08       0.01        0.05     297.00   
187  WINCHMORE HILL EXPRES       8.14     

In [118]:
regShopSalesFromFuelVsAppPriceADO_pct['label']= 'shopSalesFromFuelADO'
regShopSalesFromFuelVsAppPriceUL95_pct['label']= 'shopSalesFromFuelUL95'
regExclShopSalesVsAppPriceADO_pct['label'] = 'exclShopSalesADO'
regExclShopSalesVsAppPriceUL95_pct['label'] = 'exclShopSalesUL95'

# shpSales_el = pd.concat([regShopSalesFromFuelVsAppPriceADO_pct,regShopSalesFromFuelVsAppPriceUL95_pct,\
#           regExclShopSalesVsAppPriceADO_pct,regExclShopSalesVsAppPriceUL95_pct],axis=0)


In [126]:
# shpSales_el = pd.concat([regShopSalesFromFuelVsAppPriceADO_pct,regShopSalesFromFuelVsAppPriceUL95_pct],\
#                         axis=1,ignore_index=True)
shpSales_el = regShopSalesFromFuelVsAppPriceADO_pct.append(regShopSalesFromFuelVsAppPriceUL95_pct,ignore_index=True)

In [130]:
pd.concat([regShopSalesFromFuelVsAppPriceADO_pct,regShopSalesFromFuelVsAppPriceUL95_pct])

Site Name  count_ADO  count_UL95                  label  \
0    ABBEY ST ACCRNGTN EXP     302.00         nan   shopSalesFromFuelADO   
1         ABINGTON EXPRESS     309.00         nan   shopSalesFromFuelADO   
2    ACKLAM RD MIDSBRO EXP     282.00         nan   shopSalesFromFuelADO   
3            ACTON EXPRESS     248.00         nan   shopSalesFromFuelADO   
4       ADDISCOMBE EXPRESS     301.00         nan   shopSalesFromFuelADO   
5          ARDSLEY EXPRESS     290.00         nan   shopSalesFromFuelADO   
6        ARDWICK GREEN EXP     342.00         nan   shopSalesFromFuelADO   
7    ASHTON IN MAKRFLD EXP     260.00         nan   shopSalesFromFuelADO   
8           AULD CROSS EXP     299.00         nan   shopSalesFromFuelADO   
9           BALHAM EXPRESS     303.00         nan   shopSalesFromFuelADO   
10         BARKING EXPRESS     275.00         nan   shopSalesFromFuelADO   
11          BARNES EXPRESS     311.00         nan   shopSalesFromFuelADO   
12         BASFORD EXPRESS     282.00         nan   shopSalesFromFuelADO   
13          BATTLE EXPRESS     133.00         nan   shopSalesFromFuelADO   
14        BEARSTED EXPRESS     322.00         nan   shopSalesFromFuelADO   
15   BESSACARR DONCAST EXP     300.00         nan   shopSalesFromFuelADO   
16   BEWDLEY HILL KIDD EXP     322.00         nan   shopSalesFromFuelADO   
17      BILLY MILL EXPRESS     315.00         nan   shopSalesFromFuelADO   
18        BITTERNE EXPRESS     283.00         nan   shopSalesFromFuelADO   
19   BOLTON BRADFRD ST EXP     290.00         nan   shopSalesFromFuelADO   
20   BOOTHFERRY HESSLE EXP     350.00         nan   shopSalesFromFuelADO   
21          BOURNE EXPRESS     289.00         nan   shopSalesFromFuelADO   
22     BOURNVILLE BHAM EXP     297.00         nan   shopSalesFromFuelADO   
23   BRAMHALL CHESHIRE EXP     275.00         nan   shopSalesFromFuelADO   
24         BRECHIN EXPRESS     249.00         nan   shopSalesFromFuelADO   
25       BRENTWOOD EXPRESS     307.00         nan   shopSalesFromFuelADO   
26     BRISTL SOUNDWEL EXP     304.00         nan   shopSalesFromFuelADO   
27      BRISTOL FILTON EXP     320.00         nan   shopSalesFromFuelADO   
28         BROMLEY EXPRESS     301.00         nan   shopSalesFromFuelADO   
29   BULLSMRE LN WAL X EXP     309.00         nan   shopSalesFromFuelADO   
..                     ...        ...         ...                    ...   
166    STORRINGTON EXPRESS        nan      337.00  shopSalesFromFuelUL95   
167  TARBOCK WOODLANDS EXP        nan      319.00  shopSalesFromFuelUL95   
168       TOLWORTH EXPRESS        nan      306.00  shopSalesFromFuelUL95   
169       TONBRIDGE RD EXP        nan      322.00  shopSalesFromFuelUL95   
170        TORQUAY EXPRESS        nan      304.00  shopSalesFromFuelUL95   
171      TULSEHILL EXPRESS        nan      317.00  shopSalesFromFuelUL95   
172        TUPSLEY EXPRESS        nan      306.00  shopSalesFromFuelUL95   
173  TWYFORD RD EASTLG EXP        nan      312.00  shopSalesFromFuelUL95   
174    WALTHAMSTOW EXPRESS        nan      311.00  shopSalesFromFuelUL95   
175            WARMLEY EXP        nan      317.00  shopSalesFromFuelUL95   
176    WEDNESFIELD EXPRESS        nan      315.00  shopSalesFromFuelUL95   
177   WEST BYFLEET EXPRESS        nan      216.00  shopSalesFromFuelUL95   
178   WEST WICKHAM EXPRESS        nan      311.00  shopSalesFromFuelUL95   
179           WESTBURN EXP        nan      299.00  shopSalesFromFuelUL95   
180  WEYMOUTH LANEH RD EXP        nan      316.00  shopSalesFromFuelUL95   
181     WHITCHURCH EXPRESS        nan      299.00  shopSalesFromFuelUL95   
182     WHITEGATES EXPRESS        nan      297.00  shopSalesFromFuelUL95   
183  WHITEHOUS SCOLDFD EXP        nan      320.00  shopSalesFromFuelUL95   
184    WIGAN RD BOLTON EXP        nan      333.00  shopSalesFromFuelUL95   
185    WILLENHALL BHAM EXP        nan      311.00  shopSalesFromFuelUL95   
186      WIMBLEDON EXPRESS        nan      297.00  shopSalesFromF

In [131]:
regShopSalesFromFuelVsAppPriceADO_pct.to_excel('regShopSalesFromFuelVsAppPriceADO_pct.xlsx',index=False)
regShopSalesFromFuelVsAppPriceUL95_pct.to_excel('regShopSalesFromFuelVsAppPriceUL95_pct.xlsx',index=False)
regExclShopSalesVsAppPriceADO_pct.to_excel('regExclShopSalesVsAppPriceADO_pct.xlsx',index=False)
regExclShopSalesVsAppPriceUL95_pct.to_excel('regExclShopSalesVsAppPriceUL95_pct.xlsx',index=False)

In [127]:
shpSales_el

Site Name  count_ADO  count_UL95                  label  \
0    ABBEY ST ACCRNGTN EXP     302.00         nan   shopSalesFromFuelADO   
1         ABINGTON EXPRESS     309.00         nan   shopSalesFromFuelADO   
2    ACKLAM RD MIDSBRO EXP     282.00         nan   shopSalesFromFuelADO   
3            ACTON EXPRESS     248.00         nan   shopSalesFromFuelADO   
4       ADDISCOMBE EXPRESS     301.00         nan   shopSalesFromFuelADO   
5          ARDSLEY EXPRESS     290.00         nan   shopSalesFromFuelADO   
6        ARDWICK GREEN EXP     342.00         nan   shopSalesFromFuelADO   
7    ASHTON IN MAKRFLD EXP     260.00         nan   shopSalesFromFuelADO   
8           AULD CROSS EXP     299.00         nan   shopSalesFromFuelADO   
9           BALHAM EXPRESS     303.00         nan   shopSalesFromFuelADO   
10         BARKING EXPRESS     275.00         nan   shopSalesFromFuelADO   
11          BARNES EXPRESS     311.00         nan   shopSalesFromFuelADO   
12         BASFORD EXPRESS     282.00         nan   shopSalesFromFuelADO   
13          BATTLE EXPRESS     133.00         nan   shopSalesFromFuelADO   
14        BEARSTED EXPRESS     322.00         nan   shopSalesFromFuelADO   
15   BESSACARR DONCAST EXP     300.00         nan   shopSalesFromFuelADO   
16   BEWDLEY HILL KIDD EXP     322.00         nan   shopSalesFromFuelADO   
17      BILLY MILL EXPRESS     315.00         nan   shopSalesFromFuelADO   
18        BITTERNE EXPRESS     283.00         nan   shopSalesFromFuelADO   
19   BOLTON BRADFRD ST EXP     290.00         nan   shopSalesFromFuelADO   
20   BOOTHFERRY HESSLE EXP     350.00         nan   shopSalesFromFuelADO   
21          BOURNE EXPRESS     289.00         nan   shopSalesFromFuelADO   
22     BOURNVILLE BHAM EXP     297.00         nan   shopSalesFromFuelADO   
23   BRAMHALL CHESHIRE EXP     275.00         nan   shopSalesFromFuelADO   
24         BRECHIN EXPRESS     249.00         nan   shopSalesFromFuelADO   
25       BRENTWOOD EXPRESS     307.00         nan   shopSalesFromFuelADO   
26     BRISTL SOUNDWEL EXP     304.00         nan   shopSalesFromFuelADO   
27      BRISTOL FILTON EXP     320.00         nan   shopSalesFromFuelADO   
28         BROMLEY EXPRESS     301.00         nan   shopSalesFromFuelADO   
29   BULLSMRE LN WAL X EXP     309.00         nan   shopSalesFromFuelADO   
..                     ...        ...         ...                    ...   
362    STORRINGTON EXPRESS        nan      337.00  shopSalesFromFuelUL95   
363  TARBOCK WOODLANDS EXP        nan      319.00  shopSalesFromFuelUL95   
364       TOLWORTH EXPRESS        nan      306.00  shopSalesFromFuelUL95   
365       TONBRIDGE RD EXP        nan      322.00  shopSalesFromFuelUL95   
366        TORQUAY EXPRESS        nan      304.00  shopSalesFromFuelUL95   
367      TULSEHILL EXPRESS        nan      317.00  shopSalesFromFuelUL95   
368        TUPSLEY EXPRESS        nan      306.00  shopSalesFromFuelUL95   
369  TWYFORD RD EASTLG EXP        nan      312.00  shopSalesFromFuelUL95   
370    WALTHAMSTOW EXPRESS        nan      311.00  shopSalesFromFuelUL95   
371            WARMLEY EXP        nan      317.00  shopSalesFromFuelUL95   
372    WEDNESFIELD EXPRESS        nan      315.00  shopSalesFromFuelUL95   
373   WEST BYFLEET EXPRESS        nan      216.00  shopSalesFromFuelUL95   
374   WEST WICKHAM EXPRESS        nan      311.00  shopSalesFromFuelUL95   
375           WESTBURN EXP        nan      299.00  shopSalesFromFuelUL95   
376  WEYMOUTH LANEH RD EXP        nan      316.00  shopSalesFromFuelUL95   
377     WHITCHURCH EXPRESS        nan      299.00  shopSalesFromFuelUL95   
378     WHITEGATES EXPRESS        nan      297.00  shopSalesFromFuelUL95   
379  WHITEHOUS SCOLDFD EXP        nan      320.00  shopSalesFromFuelUL95   
380    WIGAN RD BOLTON EXP        nan      333.00  shopSalesFromFuelUL95   
381    WILLENHALL BHAM EXP        nan      311.00  shopSalesFromFuelUL95   
382      WIMBLEDON EXPRESS        nan      297.00  shopSalesFromF

In [108]:
shpSales_el.to_excel(r'elasticity_shopSales_regression_3.xlsx',index=False)

In [113]:
shpSales_el.head()

Site Name  count_ADO  count_UL95                 label  \
0  ABBEY ST ACCRNGTN EXP     302.00         nan  shopSalesFromFuelADO   
1       ABINGTON EXPRESS     309.00         nan  shopSalesFromFuelADO   
2  ACKLAM RD MIDSBRO EXP     282.00         nan  shopSalesFromFuelADO   
3          ACTON EXPRESS     248.00         nan  shopSalesFromFuelADO   
4     ADDISCOMBE EXPRESS     301.00         nan  shopSalesFromFuelADO   

   pvalue_ADO  pvalue_UL95  rvalu_ADO  rvalu_UL95  slope_ADO  slope_UL95  
0        0.45          nan       0.00         nan      -0.68         nan  
1        0.08          nan       0.01         nan      -4.83         nan  
2        0.83          nan       0.00         nan      -0.25         nan  
3        0.05          nan       0.02         nan       2.08         nan  
4        0.91          nan       0.00         nan       0.11         nan

In [75]:
el_df_ado = regFuelSalesVsAppPrice_pdi.loc[:,['Site Name', 'slope_ADO','el_levels_ADO','EL ADO','Elasticity ADO']]

In [76]:
el_df_ado.head()

Site Name  slope_ADO el_levels_ADO  EL ADO Elasticity ADO
0  ABBEY ST ACCRNGTN EXP      -1.95        Medium   -7.00     3 - Medium
1       ABINGTON EXPRESS      -1.75        Medium   -5.00   5 - Very Low
2  ACKLAM RD MIDSBRO EXP      -2.51          High   -5.00   5 - Very Low
3          ACTON EXPRESS       0.31      positive   -7.00     3 - Medium
4     ADDISCOMBE EXPRESS      -1.66        Medium   -7.00     3 - Medium

In [77]:
min_el_ado = el_df_ado.slope_ADO.min()
max_el_ado = el_df_ado.slope_ADO.max()
range_el_ado = max_el_ado - min_el_ado

min_el_ado_pdi = el_df_ado['EL ADO'].min()
max_el_ado_pdi = el_df_ado['EL ADO'].max()
range_el_ado_pdi = max_el_ado_pdi - min_el_ado_pdi

el_df_ado['el_ado_scd'] =  round((el_df_ado.slope_ADO - min_el_ado)*range_el_ado_pdi/range_el_ado + min_el_ado_pdi)

df_com = el_df_ado[['el_ado_scd','EL ADO']].dropna()

In [100]:
pd.crosstab(df_com['el_ado_scd'],df_com['EL ADO'],rownames =['Reg results'],colnames=['PDI results EL'],margins=True)

PDI results EL  -9.0  -8.0  -7.0  -6.0  -5.0  All
Reg results                                      
-9.0               1     0     0     0     0    1
-8.0               0     3    10    18    17   48
-7.0               0     3    16    58    65  142
-6.0               0     0     0     2     0    2
-5.0               0     0     0     1     0    1
All                1     6    26    79    82  194

In [94]:
match_case_pct = ((df_com['el_ado_scd']==df_com['EL ADO']).sum()+(df_com['el_ado_scd']==df_com['EL ADO']-1).sum()+\
            (df_com['el_ado_scd']==df_com['EL ADO']+1).sum())*100/len(df_com)

In [95]:
match_case_pct

48.45360824742268

In [91]:
(df_com['el_ado_scd']==df_com['EL ADO']).sum()

22

In [92]:
(df_com['el_ado_scd']==df_com['EL ADO']-1).sum()

68

In [93]:
(df_com['el_ado_scd']==df_com['EL ADO']+1).sum()

4

In [103]:
el_df_ado.to_excel(r'comparison_pdi_vs_reg.xlsx',index=False)

In [111]:
el_df_ado.columns

Index(['Site Name', 'slope_ADO', 'el_levels_ADO', 'EL ADO', 'Elasticity ADO',
       'el_ado_scd'],
      dtype='object')

In [112]:
el_df_ado.slope_ADO.median()

-1.1126054036577353

In [132]:
data.columns

Index(['Site Number', 'Site Name', 'Transaction Date', 'Total Sales Value',
       'Total Baskets', 'Sales Baskets With Fuel', 'Baskets With Fuel',
       'Site ID', 'PBL', 'CRUSO_priceADO', 'CRUSO_priceUL95', 'volumeADO',
       'volumeUL95', 'sales_valADO', 'sales_valUL95', 'totalSalesEsso',
       'pct_dev', 'diff_shop_esso', 'SiteCode', 'SiteName', 'Date',
       'UL 95_ Sales Transaction Volume', 'UL 95_ Approved Price',
       'UL 95_ Marker Price', 'UL 95_ Approved Price Vs Marker Price',
       'ADO_ Sales Transaction Volume', 'ADO_ Approved Price',
       'ADO_ Marker Price', 'ADO_ Approved Price Vs Marker Price',
       'avgUL95VolSite', 'avgADOVolSite', 'diffAdoVol', 'diffUL95Vol',
       'diffAdoVolpct', 'diffUL95Volpct', 'diffAdoPricePct',
       'diffUL95PricePct', 'avgShopSalesFromFuelSiteWise',
       'pctChgShopSalesFromFuel', 'shopSales', 'avgShopSalesSiteWise',
       'ShopSalesPctChgFromAvg', 'AdoApprPriceChgPct', 'UL95AppPriceChgPct',
       'avgFuelSalesEssoSiteWi

In [136]:
data['medianADOVolSite'] = data.groupby('PBL')['volumeADO'].transform('median')
data['medianUL95VolSite'] = data.groupby('PBL')['volumeUL95'].transform('median')
data['ADO_vol_chg'] = (data.volumeADO/data.medianADOVolSite - 1)*100
data['UL95_vol_chg'] = (data.volumeUL95/data.medianUL95VolSite - 1)*100

In [143]:
# fuel sales
x_var = 'AdoApprPriceChgPct'
ref = '_ADO'
regFuelVolEl_ADO = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'ADO_vol_chg',ref)).reset_index()

In [145]:
regFuelVolEl_ADO.mean()

slope_ADO     -1.91
rvalu_ADO      0.04
pvalue_ADO     0.17
count_ADO    294.81
dtype: float64

In [166]:
logVol = np.log10(data['volumeADO'])
logP_by_P0 = np.log10(data['ADO_ Approved Price']/data['ADO_ Marker Price'])

P_by_P0 = (data['ADO_ Approved Price']/data['ADO_ Marker Price'])

log_P_by_P0 = np.log10(P_by_P0)

data['log_P_by_P0'] = log_P_by_P0

data['logVol'] = logVol

 np.log10(data['volumeADO'])

data['volumeADO'].describe()

data2 = data.query('volumeADO >=100')

data2.loc[:,'logVolAdo'] =   np.log10(data2.loc[:,'volumeADO'])
data2['logP_by_P0'] = np.log10(data['ADO_ Approved Price']/data['ADO_ Marker Price'])

data2['log_v_by_v0'] = np.log10(data2.volumeADO/data2.medianADOVolSite)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':


In [204]:
#elasticity calculation :: forcing the intercept to zero 
def regRes_gen_const(site,x,y,prefix):
    cols = ['const','slope','rvalu','pvalue','count']
    cols_ref = list(map(lambda x: x+prefix,cols))
    #print(site.head(3))
    try:
        # filter all data of the site between 5th & 95th precentile 
        q_low,q_up = site[y].quantile([0.02,0.98]).values.round().tolist()
        xf = site.loc[(site[y] >= q_low)& (site[y] <= q_up)]

        q_low,q_up = site[x].quantile([0.02,0.98]).values.round().tolist()
        xf = site.loc[(site[x] >= q_low)& (site[x] <= q_up)]

        x_train,y_train = xf[x], xf[y]
        x_train = sm.add_constant(x_train)

        model = sm.OLS(y_train,x_train).fit()
        const,slope,rvalu,pvalue = model.params[0],model.params[1],round(model.rsquared,2),model.pvalues[1]
        #slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf[x],xf[y])

        return(pd.Series([const,slope,rvalu,pvalue,len(xf)],index=cols_ref))
    except Exception as e:
        print(e)
        return(pd.Series([0,0,0,0,len(xf)],index=cols_ref))

In [210]:
# fuel sales
x_var = 'AdoApprPriceChgPct'
ref = '_ADO'
data3 = data2.query('logP_by_P0 != 0')
regFuelVolEl_ADO = data3.groupby('Site Name').apply(lambda site: \
                                        regRes_gen_const(site,x_var,'logVolAdo',ref)).reset_index()

In [211]:
data3.logP_by_P0.describe()

count   58,344.00
mean         0.00
std          0.01
min         -0.03
25%         -0.00
50%          0.00
75%          0.01
max          0.05
Name: logP_by_P0, dtype: float64

In [208]:
regFuelVolEl_ADO

Site Name  const_ADO  slope_ADO  rvalu_ADO  pvalue_ADO  \
0    ABBEY ST ACCRNGTN EXP      -0.92       3.49       0.03        0.00   
1         ABINGTON EXPRESS       2.18       0.23       0.00        0.90   
2    ACKLAM RD MIDSBRO EXP       2.52      -1.63       0.02        0.02   
3            ACTON EXPRESS      -1.63       2.13       0.04        0.01   
4       ADDISCOMBE EXPRESS       0.67       1.21       0.00        0.32   
5          ARDSLEY EXPRESS       0.50       1.56       0.01        0.18   
6        ARDWICK GREEN EXP      -6.31       1.75       0.01        0.05   
7    ASHTON IN MAKRFLD EXP       1.30      -1.87       0.02        0.05   
8           AULD CROSS EXP       0.83      -0.07       0.00        0.91   
9           BALHAM EXPRESS       0.25       1.54       0.01        0.20   
10         BARKING EXPRESS       2.32      -4.12       0.03        0.00   
11          BARNES EXPRESS       0.32       2.18       0.02        0.04   
12         BASFORD EXPRESS       0.89      -0.15       0.00        0.77   
13          BATTLE EXPRESS       0.09       2.55       0.03        0.07   
14        BEARSTED EXPRESS       2.78       0.46       0.00        0.49   
15   BESSACARR DONCAST EXP       3.51      -1.08       0.02        0.02   
16   BEWDLEY HILL KIDD EXP       5.74      -1.80       0.01        0.04   
17      BILLY MILL EXPRESS       6.20      -4.59       0.04        0.00   
18        BITTERNE EXPRESS      -1.48       5.43       0.05        0.00   
19   BOLTON BRADFRD ST EXP       4.10      -2.41       0.02        0.01   
20   BOOTHFERRY HESSLE EXP       1.88       1.34       0.01        0.08   
21          BOURNE EXPRESS       2.87      -0.77       0.00        0.67   
22     BOURNVILLE BHAM EXP       3.18      -2.50       0.02        0.02   
23   BRAMHALL CHESHIRE EXP       1.24      -1.98       0.00        0.46   
24         BRECHIN EXPRESS       4.90      -2.05       0.02        0.02   
25       BRENTWOOD EXPRESS       1.57       0.47       0.00        0.54   
26     BRISTL SOUNDWEL EXP       1.67      -0.07       0.00        0.92   
27      BRISTOL FILTON EXP      -0.83       1.00       0.01        0.14   
28         BROMLEY EXPRESS       1.75      -0.08       0.00        0.94   
29   BULLSMRE LN WAL X EXP       0.54      -0.24       0.00        0.80   
..                     ...        ...        ...        ...         ...   
166    STORRINGTON EXPRESS       1.65      -0.98       0.01        0.19   
167  TARBOCK WOODLANDS EXP      -4.03       5.85       0.17        0.00   
168       TOLWORTH EXPRESS       2.29      -1.37       0.00        0.29   
169       TONBRIDGE RD EXP      -0.06       0.10       0.00        0.92   
170        TORQUAY EXPRESS      -0.76       0.10       0.00        0.96   
171      TULSEHILL EXPRESS       3.28      -0.50       0.00        0.63   
172        TUPSLEY EXPRESS       2.22      -2.49       0.02        0.01   
173  TWYFORD RD EASTLG EXP       1.16       0.12       0.00        0.72   
174    WALTHAMSTOW EXPRESS       0.91      -1.76       0.01        0.07   
175            WARMLEY EXP       0.12       0.67       0.00        0.32   
176    WEDNESFIELD EXPRESS      -0.47       1.85       0.01        0.14   
177   WEST BYFLEET EXPRESS      -5.61       6.88       0.12        0.00   
178   WEST WICKHAM EXPRESS       1.02       0.20       0.00        0.76   
179           WESTBURN EXP      -3.80      -1.62       0.01        0.06   
180  WEYMOUTH LANEH RD EXP       3.49      -0.66       0.00        0.59   
181     WHITCHURCH EXPRESS       1.95       0.24       0.00        0.70   
182     WHITEGATES EXPRESS       1.73       0.40       0.00        0.72   
183  WHITEHOUS SCOLDFD EXP       1.36      -0.39       0.00        0.57   
184    WIGAN RD BOLTON EXP       1.20      -0.19       0.00        0.80   
185    WILLENHALL BHAM EXP      -5.27       5.33       0.04        0.00   
186      WIMBLEDON EXPRESS      -0.64      -4.36       0.06        0.00   
187  WINCHMORE HILL EXPRES       1.42     

In [206]:
# calculation with respect to ADO price position 
# exclusive shop sales 
x_var = 'AdoApprPriceChgPct'
ref = '_ADO'
regExclShopSalesVsAppPriceADO_pct = data.groupby('Site Name')\
            .apply(lambda site: regRes_gen_no_const(site,x_var,'pctChgExclShopSalesFromMedian',ref)).reset_index()

# fuel sales
x_var = 'logP_by_P0'
ref = '_ADO'
data3 = data2.query('logP_by_P0 != 0')
regFuelVolEl_ADO_2 = data3.groupby('Site Name').apply(lambda site: \
                                        regRes_gen_const(site,x_var,'ADO_vol_chg',ref)).reset_index()

Site Name  const_ADO  slope_ADO  rvalu_ADO  pvalue_ADO  \
0    ABBEY ST ACCRNGTN EXP          0          0          0           0   
1         ABINGTON EXPRESS          0          0          0           0   
2    ACKLAM RD MIDSBRO EXP          0          0          0           0   
3            ACTON EXPRESS          0          0          0           0   
4       ADDISCOMBE EXPRESS          0          0          0           0   
5          ARDSLEY EXPRESS          0          0          0           0   
6        ARDWICK GREEN EXP          0          0          0           0   
7    ASHTON IN MAKRFLD EXP          0          0          0           0   
8           AULD CROSS EXP          0          0          0           0   
9           BALHAM EXPRESS          0          0          0           0   
10         BARKING EXPRESS          0          0          0           0   
11          BARNES EXPRESS          0          0          0           0   
12         BASFORD EXPRESS          0          0          0           0   
13          BATTLE EXPRESS          0          0          0           0   
14        BEARSTED EXPRESS          0          0          0           0   
15   BESSACARR DONCAST EXP          0          0          0           0   
16   BEWDLEY HILL KIDD EXP          0          0          0           0   
17      BILLY MILL EXPRESS          0          0          0           0   
18        BITTERNE EXPRESS          0          0          0           0   
19   BOLTON BRADFRD ST EXP          0          0          0           0   
20   BOOTHFERRY HESSLE EXP          0          0          0           0   
21          BOURNE EXPRESS          0          0          0           0   
22     BOURNVILLE BHAM EXP          0          0          0           0   
23   BRAMHALL CHESHIRE EXP          0          0          0           0   
24         BRECHIN EXPRESS          0          0          0           0   
25       BRENTWOOD EXPRESS          0          0          0           0   
26     BRISTL SOUNDWEL EXP          0          0          0           0   
27      BRISTOL FILTON EXP          0          0          0           0   
28         BROMLEY EXPRESS          0          0          0           0   
29   BULLSMRE LN WAL X EXP          0          0          0           0   
..                     ...        ...        ...        ...         ...   
166    STORRINGTON EXPRESS          0          0          0           0   
167  TARBOCK WOODLANDS EXP          0          0          0           0   
168       TOLWORTH EXPRESS          0          0          0           0   
169       TONBRIDGE RD EXP          0          0          0           0   
170        TORQUAY EXPRESS          0          0          0           0   
171      TULSEHILL EXPRESS          0          0          0           0   
172        TUPSLEY EXPRESS          0          0          0           0   
173  TWYFORD RD EASTLG EXP          0          0          0           0   
174    WALTHAMSTOW EXPRESS          0          0          0           0   
175            WARMLEY EXP          0          0          0           0   
176    WEDNESFIELD EXPRESS          0          0          0           0   
177   WEST BYFLEET EXPRESS          0          0          0           0   
178   WEST WICKHAM EXPRESS          0          0          0           0   
179           WESTBURN EXP          0          0          0           0   
180  WEYMOUTH LANEH RD EXP          0          0          0           0   
181     WHITCHURCH EXPRESS          0          0          0           0   
182     WHITEGATES EXPRESS          0          0          0           0   
183  WHITEHOUS SCOLDFD EXP          0          0          0           0   
184    WIGAN RD BOLTON EXP          0          0          0           0   
185    WILLENHALL BHAM EXP          0          0          0           0   
186      WIMBLEDON EXPRESS          0          0          0           0   
187  WINCHMORE HILL EXPRES          0     

In [197]:
regFuelVolEl_ADO

Site Name  const_ADO  slope_ADO  rvalu_ADO  pvalue_ADO  \
0    ABBEY ST ACCRNGTN EXP       1.72      -4.04       0.03        0.00   
1         ABINGTON EXPRESS      -0.84      -2.60       0.01        0.17   
2    ACKLAM RD MIDSBRO EXP       2.12      -3.60       0.08        0.00   
3            ACTON EXPRESS      -6.57       0.83       0.00        0.60   
4       ADDISCOMBE EXPRESS       0.35      -3.47       0.01        0.11   
5          ARDSLEY EXPRESS      -1.70      -4.92       0.04        0.00   
6        ARDWICK GREEN EXP      -9.86       0.42       0.00        0.78   
7    ASHTON IN MAKRFLD EXP      -2.52      -0.88       0.00        0.36   
8           AULD CROSS EXP      -4.34      -3.81       0.04        0.00   
9           BALHAM EXPRESS      -2.30       2.31       0.01        0.16   
10         BARKING EXPRESS      -1.76      -0.60       0.00        0.73   
11          BARNES EXPRESS       4.93      14.00       0.04        0.00   
12         BASFORD EXPRESS      -0.34      -2.36       0.03        0.00   
13          BATTLE EXPRESS      -4.38       0.97       0.00        0.62   
14        BEARSTED EXPRESS      -0.30      -5.98       0.14        0.00   
15   BESSACARR DONCAST EXP      -1.50       1.20       0.03        0.00   
16   BEWDLEY HILL KIDD EXP       5.74      -4.15       0.07        0.00   
17      BILLY MILL EXPRESS       2.40      -4.91       0.07        0.00   
18        BITTERNE EXPRESS      -0.42      -1.58       0.00        0.42   
19   BOLTON BRADFRD ST EXP      -0.92      -0.51       0.00        0.64   
20   BOOTHFERRY HESSLE EXP      -0.49      -5.28       0.07        0.00   
21          BOURNE EXPRESS       2.88      -5.56       0.04        0.00   
22     BOURNVILLE BHAM EXP       0.44      -3.12       0.01        0.05   
23   BRAMHALL CHESHIRE EXP      -6.23      -7.17       0.02        0.03   
24         BRECHIN EXPRESS      -3.91      -0.25       0.00        0.76   
25       BRENTWOOD EXPRESS      -1.98      -2.08       0.01        0.06   
26     BRISTL SOUNDWEL EXP      -5.05       1.05       0.00        0.36   
27      BRISTOL FILTON EXP      -1.42       0.12       0.00        0.91   
28         BROMLEY EXPRESS      -0.47      -3.03       0.02        0.04   
29   BULLSMRE LN WAL X EXP      -3.38       0.35       0.00        0.80   
..                     ...        ...        ...        ...         ...   
166    STORRINGTON EXPRESS       0.70      -5.80       0.09        0.00   
167  TARBOCK WOODLANDS EXP      -7.22       3.96       0.08        0.00   
168       TOLWORTH EXPRESS       0.48      -1.94       0.01        0.28   
169       TONBRIDGE RD EXP      -0.87      -1.77       0.01        0.15   
170        TORQUAY EXPRESS      -0.19      -5.96       0.04        0.00   
171      TULSEHILL EXPRESS       0.95      -2.78       0.01        0.09   
172        TUPSLEY EXPRESS      -0.58      -3.29       0.02        0.01   
173  TWYFORD RD EASTLG EXP      -4.23      -3.70       0.13        0.00   
174    WALTHAMSTOW EXPRESS      -1.30      -0.05       0.00        0.98   
175            WARMLEY EXP     -13.64       4.95       0.05        0.00   
176    WEDNESFIELD EXPRESS      -0.31      -1.58       0.00        0.33   
177   WEST BYFLEET EXPRESS      -0.48      -1.98       0.00        0.32   
178   WEST WICKHAM EXPRESS      -1.32      -3.95       0.05        0.00   
179           WESTBURN EXP      -9.07      -5.18       0.04        0.00   
180  WEYMOUTH LANEH RD EXP       2.37      -5.29       0.03        0.00   
181     WHITCHURCH EXPRESS      -1.67       0.31       0.00        0.66   
182     WHITEGATES EXPRESS       1.81      -3.42       0.02        0.03   
183  WHITEHOUS SCOLDFD EXP       0.59      -0.56       0.00        0.53   
184    WIGAN RD BOLTON EXP      -0.54       2.43       0.01        0.04   
185    WILLENHALL BHAM EXP      -0.45       0.28       0.00        0.85   
186      WIMBLEDON EXPRESS      -3.32     -10.50       0.10        0.00   
187  WINCHMORE HILL EXPRES      -2.15     